In [ ]:
# Импортируем библиотеки
import os
import pandas as pd
import seaborn as sn
import numpy as np
import dask.dataframe as dd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering


In [ ]:
df = pd.read_csv(f'data_year.csv', sep=',')
df

In [ ]:
# Recency
day = '2022-01-01' # Начальная дата - самая ранняя
day = pd.to_datetime(day)
df['date'] =  pd.to_datetime(df['date']).dt.normalize()
dates = df['date'].to_list()
df = df.assign(date = -1 * (day - df['date'])) # Берем начальную и вычитаем из нее максимальную дату 
df['date'] = [-1*(day - date).days for date in dates]
recency = df['date']
recency

In [ ]:
# Frequency
freq = df['qw_count']
freq

In [ ]:
# Monetary value

money = df['amount_sum']
money

In [ ]:
recency.columns=['Recency']
freq.columns=['Frequency']
money.columns=['Monetary']
RFM = pd.concat([recency, freq, money], axis=1)
RFM

In [ ]:
RFM[RFM.isnull().any(1)]

In [ ]:
df.dropna()

In [ ]:
# Машинное обучение
scaler = StandardScaler()
scaled = scaler.fit_transform(RFM)

# Выбираем количество кластеров
# inertia = []
# for i in np.arange(1, 11):
#     kmeans = KMeans(n_clusters=i, init = 'k-means++')
#     kmeans.fit(scaled)
#     inertia.append(kmeans.inertia_)

# plt.plot(inertia, marker='o')

In [ ]:
# Из графика видно, что нам нужно 3 Кластера
kmeans = KMeans(n_clusters=3)
kmeans.fit(scaled)
df['Clusters'] = (kmeans.labels_ + 1)
df

In [ ]:
df = df.drop(columns=['qw_count', 'amount_sum', 'txn_id_sum', 'revenue', 'costs', 'date'])

df.to_csv('clusters.csv')

In [ ]:
df = df.assign(average_check=df['amount_sum']/df['qw_count'])
df.to_csv('df_clusters.csv')

In [ ]:
group = df.groupby(['Clusters']).agg({
        'qw_count': 'sum',
        'amount_sum': 'sum',
        'txn_id_sum': 'sum',
        'revenue': 'sum',
        'costs': 'sum',
        'date': 'mean'
    })
group

In [ ]:
pd.options.display.float_format ='{:,.8f}'.format
RFM['Clusters'] = (kmeans.labels_ + 1)
RFM_group = RFM.groupby(['Clusters'])['date', 'qw_count', 'amount_sum'].mean()
RFM_group

In [ ]:
group = group.assign(average_check=group['amount_sum']/group['qw_count'])
group

In [ ]:
df

In [ ]:
quantity_users = df.groupby(['Clusters'])['qw'].count()
quantity_users

In [ ]:
group['quantity_users'] = quantity_users
group

In [ ]:
registration = pd.read_csv(f'dataset/reg_data/qw_regs.csv', sep=';')
registration


In [ ]:
df_new_users = pd.concat([df, registration], axis=1)
df_new_users = df_new_users.dropna()
df_new_users


In [ ]:
df_new_users['from_dt'] = pd.to_datetime(df_new_users['from_dt']).dt.normalize()
dates_reg = df_new_users['from_dt'].to_list()
day = pd.to_datetime('2022-01-01')
spisok = []
for d in dates_reg:
    if (day - d).days > 0:
        spisok.append(0)
    else:
        spisok.append(1)

print(spisok)


In [ ]:
df_new_users['is_new'] = spisok
df_new_users

In [ ]:
df_new_users_group = df_new_users.groupby(['Clusters']).agg({
        'qw_count': 'sum',
        'amount_sum': 'sum',
        'txn_id_sum': 'sum',
        'revenue': 'sum',
        'costs': 'sum',
        'average_check': 'mean',
        'date': 'mean',
        'is_new': 'sum'
    })
df_new_users_group['quantity_users'] = quantity_users
df_new_users_group